In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

## Test 12.1 European Options GBSM including Greeks

In [2]:
data_12_1 = pd.read_csv("testfiles/data/test12_1.csv").dropna()
data_12_1

,ID,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,DividendRate,ImpliedVol
0,1.0,Call,100.0,100.0,50.0,365.0,0.045,0.00,0.2
1,2.0,Put,100.0,100.0,50.0,365.0,0.045,0.00,0.2
2,3.0,Call,100.0,90.0,400.0,365.0,0.045,0.02,0.4
3,4.0,Put,100.0,110.0,400.0,365.0,0.045,0.02,0.4


In [2]:
def European_GBSM(S, X, T, vol, r, b, option_type="call"):
    '''
        S - Underlying Price
        X - Strike Price
        T - is the Time to Maturity
        vol - is the implied volatilityσ
        r - is the risk free rate
        b - is the cost of carry

        black scholes where rf = b
    '''

    option_type = str.lower(option_type)
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be call or put")
        
    d1 = (np.log(S/X) + (b + (vol**2)/2) * T) / (vol * np.sqrt(T))

    d2 = d1 - vol * np.sqrt(T)

    if option_type == "call":
        value = S * np.exp((b-r)*T) * norm.cdf(d1) - X * np.exp(-r*T) * norm.cdf(d2)
        delta = np.exp((b-r)*T) * norm.cdf(d1)
        theta = - (S * np.exp((b-r)*T) * norm.pdf(d1) * vol) / (2 * np.sqrt(T)) - (b-r) * S*np.exp((b-r)*T) * norm.cdf(d1) - r*X*np.exp(-r*T) * norm.cdf(d2)

        rho = T * X * np.exp(-r*T) * norm.cdf(d2)
        carry_rho = T*S * np.exp((b-r)*T) * norm.cdf(d1) # dP / db

    else:
        value = X * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp((b-r)*T) * norm.cdf(-d1)
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
        theta = - (S * np.exp((b-r)*T) * norm.pdf(d1) * vol) / (2 * np.sqrt(T)) + (b-r) * S*np.exp((b-r)*T) * norm.cdf(-d1) + r*X*np.exp(-r*T) * norm.cdf(-d2)

        rho = -T * X * np.exp(-r*T) * norm.cdf(-d2)
        carry_rho = -T*S * np.exp((b-r)*T) * norm.cdf(-d1) # dP / db
    
    gamma = norm.pdf(d1) * np.exp((b-r)*T) / (S * vol * np.sqrt(T))
    vega = S * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    
    
    return value, delta, gamma, vega, theta, rho, carry_rho

In [4]:
data_12_1 = pd.read_csv("testfiles/data/test12_1.csv").dropna()
test_12_1 = pd.read_csv("testfiles/data/testout12_1.csv")
test_12_1 = test_12_1.set_index("ID")

error_epsilon = 1e-14
for index, row in data_12_1.iterrows():
    id = row["ID"]
    option_type = row["Option Type"]
    S = row["Underlying"]
    X = row["Strike"]
    days = row["DaysToMaturity"]
    dpy = row["DayPerYear"]
    T = days/dpy
    vol = row["ImpliedVol"]
    r = row["RiskFreeRate"]
    div = row["DividendRate"]
    b = r - div # rfr - dividend
    # b = 0
    
    value, delta, gamma, vega, theta, rho, carry_rho = European_GBSM(S, X, T, vol, r, b, option_type=option_type)
    t_value, t_delta, t_gamma, t_vega, t_rho, t_theta = test_12_1.loc[id]

    assert abs(t_value - value) < error_epsilon, f"Option id: {id} Price Does Not Match"
    assert abs(t_delta - delta) < error_epsilon, f"Option id: {id} Delta Does Not Match"
    assert abs(t_gamma - gamma) < error_epsilon, f"Option id: {id} Gamma Does Not Match"
    assert abs(t_theta - theta) < error_epsilon, f"Option id: {id} Theta Does Not Match"
    assert abs(t_vega - vega) < error_epsilon, f"Option id: {id} Vega Does Not Match"
    assert abs(t_rho - rho) < error_epsilon, f"Option id: {id} Rho Does Not Match"




In [5]:
test_12_1

,Value,Delta,Gamma,Vega,Rho,Theta
ID,,,,,,
1,3.260824,0.547872,0.053506,14.659079,7.058414,-13.019817
2,2.646281,-0.452128,0.053506,14.659079,-6.556032,-8.547471
3,22.043329,0.685508,0.008115,35.571438,50.967099,-7.213608
4,20.449083,-0.470751,0.009310,40.812329,-73.999110,-5.351164


## Test 12.2 American Options with continuous Dividends including Greeks

In [6]:
data_12_2 = pd.read_csv("testfiles/data/test12_1.csv").dropna()

In [7]:
data_12_2

,ID,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,DividendRate,ImpliedVol
0,1.0,Call,100.0,100.0,50.0,365.0,0.045,0.00,0.2
1,2.0,Put,100.0,100.0,50.0,365.0,0.045,0.00,0.2
2,3.0,Call,100.0,90.0,400.0,365.0,0.045,0.02,0.4
3,4.0,Put,100.0,110.0,400.0,365.0,0.045,0.02,0.4


In [3]:
def American_Binary_Tree(S, X, T, vol, r, b, N, option_type = "call"):
    option_type = str.lower(option_type)
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be call or put")

    dt = T/N
    u = np.exp(vol*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-r*dt)
    payoff_side = 1 if option_type=="call" else -1

    # vectorize for ease of use
    num_nodes = lambda N: int((N+1)*(N+2)/2) # 0 indexed (0th column to start)
    total_nodes = num_nodes(N)
    get_idx = lambda j, i: num_nodes(j-1)+i # 0 indexed

    option_values = [0.0]*total_nodes

    # work backward from future to present
    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = get_idx(j, i)
            option_values[idx] = max(0.0, payoff_side*(S*(u**i)*(d**(j-i)) - X))

            if j < N: # if not the final layer, we might early exercise
                option_values[idx] = max(option_values[idx], df * (pu*option_values[get_idx(j+1, i+1)] + pd*option_values[get_idx(j+1, i)]))
    
    return option_values[0]

In [38]:
def approx_delta(h, S, X, T, vol, r, b, N, option_type, richardson = True):
    def D1(h):
        v_up  = American_Binary_Tree(S + h, X, T, vol, r, b, N=N, option_type=option_type)
        v_dn  = American_Binary_Tree(S - h, X, T, vol, r, b, N=N, option_type=option_type)
        return (v_up - v_dn) / (2*h)
    
    if richardson:
        D_h = D1(h)
        D_h2 = D1(h/2)
        richardson_approx = (4*D_h2 - D_h) / 3
        return richardson_approx
    
    else: 
        approx = D1(h)
        return approx

def approx_theta(h, S, X, T, vol, r, b, N, option_type, richardson = True):
    def D1(h):
        v_up  = American_Binary_Tree(S, X, T+h, vol, r, b, N=N, option_type=option_type)
        v_dn  = American_Binary_Tree(S, X, T-h, vol, r, b, N=N, option_type=option_type)
        return (v_up - v_dn) / (2*h)
    
    if richardson:
        D_h = D1(h)
        D_h2 = D1(h/2)
        richardson_approx = (4*D_h2 - D_h) / 3
        return richardson_approx
    
    else: 
        approx = D1(h)
        return approx

def approx_vega(h, S, X, T, vol, r, b, N, option_type, richardson = True):
    def D1(h):
        v_up  = American_Binary_Tree(S, X, T, vol+h, r, b, N=N, option_type=option_type)
        v_dn  = American_Binary_Tree(S, X, T, vol-h, r, b, N=N, option_type=option_type)
        return (v_up - v_dn) / (2*h)
    
    if richardson:
        D_h = D1(h)
        D_h2 = D1(h/2)
        richardson_approx = (4*D_h2 - D_h) / 3
        return richardson_approx
    
    else: 
        approx = D1(h)
        return approx

def approx_rho(h, S, X, T, vol, r, b, N, option_type, richardson = True):
    def D1(h):
        v_up  = American_Binary_Tree(S, X, T, vol, r+h, b, N=N, option_type=option_type)
        v_dn  = American_Binary_Tree(S, X, T, vol, r-h, b, N=N, option_type=option_type)
        return (v_up - v_dn) / (2*h)
    
    if richardson:
        D_h = D1(h)
        D_h2 = D1(h/2)
        richardson_approx = (4*D_h2 - D_h) / 3
        return richardson_approx
    
    else: 
        approx = D1(h)
        return approx

def approx_gamma(h, S, X, T, vol, r, b, N, option_type, richardson = True):
    def D2(h):
        v_up  = American_Binary_Tree(S + h, X, T, vol, r, b, N=N, option_type=option_type)
        v_mid = American_Binary_Tree(S,     X, T, vol, r, b, N=N, option_type=option_type)
        v_dn  = American_Binary_Tree(S - h, X, T, vol, r, b, N=N, option_type=option_type)
        return (v_up - 2*v_mid + v_dn) / (h**2)
    
    if richardson:
        D_h   = D2(h)
        D_h2  = D2(h/2)
        richardson_approx = (4*D_h2 - D_h) / 3
        return richardson_approx
    else:
        approx = D2(h)
        return approx

In [12]:
test_12_2 = pd.read_csv("testfiles/data/testout12_2.csv")
test_12_2

,ID,Value,Delta,Gamma,Vega,Rho,Theta
0,1,3.259347,0.547849,0.055326,14.651751,-0.446486,13.014427
1,2,2.692775,-0.470676,0.057210,14.611961,-0.253731,8.939710
2,3,22.050266,0.676370,0.010698,35.904756,-22.898610,7.280144
3,4,21.019233,-0.489952,0.002654,40.759573,-14.102684,5.956511


In [40]:
data_12_2 = pd.read_csv("testfiles/data/test12_1.csv").dropna()
test_12_2 = pd.read_csv("testfiles/data/testout12_2.csv")
test_12_2 = test_12_2.set_index("ID")

error_epsilon = 3e-2
for index, row in data_12_2.iterrows():
    id = row["ID"]
    option_type = row["Option Type"]
    S = row["Underlying"]
    X = row["Strike"]
    days = row["DaysToMaturity"]
    dpy = row["DayPerYear"]
    T = days/dpy
    vol = row["ImpliedVol"]
    r = row["RiskFreeRate"]
    div = row["DividendRate"]
    b = r - div # rfr - dividend

    # NOTE -> tried doing some multiple * number of days, but it was taking too long and was wrong
    # N_multiplier = 1 # int(days*N_multiplier)
    N = 1_000
    
    value= American_Binary_Tree(S, X, T, vol, r, b, N=N, option_type=option_type)

    d_first_deriv = 0.001
    d_second_deriv = 1.0

    delta = approx_delta(d_first_deriv, S, X, T, vol, r, b, N=N, option_type=option_type, richardson=False)
    vega = approx_vega(d_first_deriv, S, X, T, vol, r, b, N=N, option_type=option_type, richardson=False)
    theta = approx_theta(d_first_deriv, S, X, T, vol, r, b, N=N, option_type=option_type, richardson=False)
    rho = approx_rho(d_first_deriv, S, X, T, vol, r, b, N=N, option_type=option_type, richardson=False)
    gamma = approx_gamma(d_second_deriv, S, X, T, vol, r, b, N=N, option_type=option_type, richardson=False)

    t_value, t_delta, t_gamma, t_vega, t_rho, t_theta = test_12_2.loc[id]
    print(index)
    print(value, t_value)
    print(delta, t_delta)
    print(gamma, t_gamma)
    print(vega, t_vega)
    print(theta, t_theta)
    print(rho, t_rho)
    assert abs(t_value - value) < error_epsilon, f"Option id: {id} Price Does Not Match"
    assert abs(t_delta - delta) < error_epsilon, f"Option id: {id} Delta Does Not Match"
    assert abs(t_gamma - gamma) < error_epsilon, f"Option id: {id} Gamma Does Not Match"
    assert abs(t_theta - theta) < error_epsilon, f"Option id: {id} Theta Does Not Match"
    assert abs(t_vega - vega) < error_epsilon, f"Option id: {id} Vega Does Not Match"
    assert abs(t_rho - rho) < error_epsilon, f"Option id: {id} Rho Does Not Match"

0
3.2600854568582527 3.2593473629162446
0.5478605790796287 0.5478487068460673
0.05479547469166901 0.0553263306434001
14.655413467307365 14.651751269603196
13.017193350312262 13.01442690694716
-0.4465870504175573 -0.4464859371991539
1
2.693227663386325 2.692774912299483
-0.46850960773370254 -0.4706758102377653
0.0567707521296863 0.057210031865994
14.614944071624558 14.611960888056414
8.940900899309545 8.939709684724102
-0.253839626310981 -0.2537305058038981
2
22.045852788568943 22.050265969030416
0.6755789402959067 0.6763702006782233
0.020263604327880103 0.0106981516565175
35.914768010707476 35.90475601672572
7.282204445568041 7.280144138661618
-22.888626208562357 -22.89861021261392
3
21.01510988654669 21.01923278329123
-0.48949378997953374 -0.4899519501407267
0.0027967898847442996 0.0026538705322265
40.76051569665751 40.759573348396245
5.9573632671892085 5.956510982965839
-14.091303647376208 -14.102684316977951


## Test 12.3 American Options with discreet Dividends

In [35]:
data_12_3 = pd.read_csv("testfiles/data/test12_3.csv")
data_12_3

,ID,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,ImpliedVol,DividendDates,DividendAmts
0,1,Call,100,100,250,365,0.045,0.4,"75,150",".01,.01"
1,2,Put,100,100,250,365,0.045,0.4,"75,150",".01,.01"


In [57]:
type(round(2.3)), round(2.8)

(int, 3)

In [49]:
def American_Binary_Tree_Discontinuous_Div(S, X, T, vol, r, div_amts: list, div_times: list, N, option_type=option_type):

    '''
        NOTE -> div_times came in data as number of days, it has been updated here to be annualized (matching T) because it makes more sense that way
                This means we have to get the indices by dividing by dt, and we need N to be a multiple of all of our div times in order for these
                to naturally be integers without rounding (though we use rounding just in case, they should be integer values by default)

        NOTE -> we were told in class that div_amts is in dollars, but that's strange since the div amounts are 0.01, which is incredibly small
    '''
    option_type = str.lower(option_type)
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be call or put")
        
    if len(div_amts) != len(div_times):
        raise IndexError("Div amounts and times must be the same size")
    
    if not div_amts or div_times[0] > T:
        return American_Binary_Tree(S, X, T, vol, r, b=r, N=N, option_type=option_type)

    dt = T/N
    div_time_indices = [round(div_time/dt) for div_time in div_times]
    u = np.exp(vol*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-r*dt)
    payoff_side = 1 if option_type=="call" else -1

    # vectorize for ease of use
    num_nodes = lambda N: int((N+1)*(N+2)/2) # 0 indexed (0th column to start)
    get_idx = lambda j, i: num_nodes(j-1)+i # 0 indexed
    num_divs = len(div_times)
    total_nodes = num_nodes(div_time_indices[0])

    option_values = [0.0]*total_nodes

    # work backward from future to present
    for j in range(div_time_indices[0], -1, -1):
        for i in range(j, -1, -1):
            idx = get_idx(j, i)
            price = S*(u**i)*(d**(j-i))

            if j < div_time_indices[0]:
                option_values[idx] = max(0.0, payoff_side*(price - X))
                option_values[idx] = max(option_values[idx], df * (pu*option_values[get_idx(j+1, i+1)] + pd*option_values[get_idx(j+1, i)]))
            else:
                no_exercise_val = American_Binary_Tree_Discontinuous_Div(price-div_amts[0], X, T-div_times[0], vol, r, div_amts[1:], [div_time - div_times[0] for div_time in div_times[1:]], N-div_time_indices[0], option_type)
                exercise_val = max(0, payoff_side*(price-X))
                option_values[idx] = max(exercise_val, no_exercise_val)
    
    return option_values[0]
    

In [52]:
data_12_3 = pd.read_csv("testfiles/data/test12_3.csv").dropna()
test_12_3 = pd.read_csv("testfiles/data/testout12_3.csv")
test_12_3 = test_12_3.set_index("ID")

error_epsilon = 5e-3
for index, row in data_12_3.iterrows():
    id = row["ID"]
    option_type = row["Option Type"]
    S = row["Underlying"]
    X = row["Strike"]
    days = row["DaysToMaturity"]
    dpy = row["DayPerYear"]
    T = days/dpy
    vol = row["ImpliedVol"]
    r = row["RiskFreeRate"]
    div_amts = list(map(float, row['DividendAmts'].split(",")))
    div_times = list(map(int, row['DividendDates'].split(",")))

    # have to do this to ensure that our dividend times are integers
    lcm_multiple = math.lcm(*div_times)
    N = lcm_multiple
    while N < 200: 
        N *= 2

    dt = T/N
    div_times = [div_time / dpy for div_time in div_times] # in years (just to keep consistent with T)

    print("STARTING", index)
    value= American_Binary_Tree_Discontinuous_Div(S, X, T, vol, r, div_amts, div_times, N=N, option_type=option_type)
    t_value = test_12_3.loc[id, "Value"]

    print(value, t_value)
    assert abs(t_value - value) < error_epsilon, f"Option id: {id} Price Does Not Match"

STARTING 0
14.49915786791836 14.503386425003823
STARTING 1
11.777170296540653 11.779631630155087
